In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
from datetime import date
import datetime

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import set_config
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [29]:
df_main = pd.read_csv('../data/train_val_data.csv')
df_test = pd.read_csv('../data/test_data.csv')

In [36]:
df_main.rename(columns = {'annual':'salary'}, inplace = True)

In [30]:
df_main.drop(['summed_annual_salary'], inplace=True, axis = 1)
df_test.drop(['summed_annual_salary'], inplace=True, axis = 1)

In [31]:
date_ref = date(2022,1,1)

In [32]:
def experience(list_date, date_base):
    exp = []
    for date_temp in list_date:
        date_res = date_base - datetime.datetime.strptime(date_temp, '%m/%d/%y').date()
        exp.append(abs(date_res.days))       
    return exp

In [33]:
xp_main = experience(list(df_main['hire_date']), date_ref)
xp_test = experience(list(df_test['hire_date']), date_ref)

In [34]:
df_main["experience"] = xp_main
df_test["experience"] = xp_test

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df_main.drop(['salary'], axis=1), df_main['salary'], test_size=.1, random_state=40)

### Dirty_cat stuff

In [39]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [59]:
from sklearn.compose import make_column_transformer
encoder = make_column_transformer(
    (one_hot, ['agency_name', 'employee_type', 'rate', 'hrswkd', 'mi', 'ethnicity', 'gender', 'experience']),
    ('passthrough', ['statenum', 'hire_date']),
    # Last but not least, our dirty column
    (one_hot, ['class_title']),
    remainder='drop',
    )

In [60]:
from sklearn.experimental import enable_hist_gradient_boosting
# now you can import the HGBR from ensemble
from sklearn.ensemble import HistGradientBoostingRegressor

# We then create a pipeline chaining our encoders to a learner
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(encoder, HistGradientBoostingRegressor())

In [61]:
pipeline.fit(X_train, y_train)

MemoryError: Unable to allocate 4.73 GiB for an array with shape (105729, 6005) and data type float64

MemoryError: Unable to allocate 4.73 GiB for an array with shape (105729, 6005) and data type float64

MemoryError: Unable to allocate 4.55 GiB for an array with shape (105729, 5779) and data type float64

In [62]:
from dirty_cat import SimilarityEncoder, TargetEncoder, MinHashEncoder, GapEncoder

encoders = {
    'one-hot': one_hot,
    'similarity': SimilarityEncoder(similarity='ngram'),
    'target': TargetEncoder(handle_unknown='ignore'),
    'minhash': MinHashEncoder(n_components=100),
    'gap': GapEncoder(n_components=100),
}

In [77]:
from sklearn.model_selection import cross_val_score

all_scores = dict()

for name, method in encoders.items():
    encoder = make_column_transformer(
        (one_hot, ['agency_name', 'ethnicity', 'gender', 'experience']),
    ('passthrough', ['statenum', 'hire_date', 'rate', 'hrswkd', 'mi', 'employee_type']),
        # Last but not least, our dirty column
        (method, ['class_title']),
        remainder='drop',
    )

In [78]:
pipeline = make_pipeline(encoder, HistGradientBoostingRegressor())
scores = cross_val_score(pipeline, X_train, y_train,error_score='raise')
print(f'{name} encoding')
print(f'r2 score:  mean: {np.mean(scores):.3f}; '
        f'std: {np.std(scores):.3f}\n')
all_scores[name] = scores

MemoryError: Unable to allocate 3.40 GiB for an array with shape (84583, 5393) and data type float64

MemoryError: Unable to allocate 3.33 GiB for an array with shape (84583, 5282) and data type float64

MemoryError: Unable to allocate 3.40 GiB for an array with shape (84583, 5393) and data type float64

MemoryError: Unable to allocate 3.40 GiB for an array with shape (84583, 5402) and data type float64

MemoryError: Unable to allocate 3.53 GiB for an array with shape (84583, 5595) and data type float64

In [ ]:
import seaborn
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3))
ax = seaborn.boxplot(data=pd.DataFrame(all_scores), orient='h')
plt.ylabel('Encoding', size=20)
plt.xlabel('Prediction accuracy     ', size=20)
plt.yticks(size=20)
plt.tight_layout()

### Super Vectorizer

In [79]:
from dirty_cat import SuperVectorizer

pipeline = make_pipeline(
    SuperVectorizer(auto_cast=True),
    HistGradientBoostingRegressor()
)

In [81]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y, scoring='r2')

print(f'scores={scores}')
print(f'mean={np.mean(scores)}')
print(f'std={np.std(scores)}')

KeyboardInterrupt: 

In [ ]:
sup_vec = SuperVectorizer(auto_cast=True)

In [ ]:
X_train_enc = sup_vec.fit_transform(X_train, y_train)
X_test_enc = sup_vec.transform(X_test)

In [ ]:
X_train_enc

In [ ]:
(X_train.shape, X_train_enc.shape)

In [ ]:
sup_vec.transformers_

In [ ]:
X.columns.to_list()

In [ ]:
feature_names = sup_vec.get_feature_names()
feature_names[:8]

In [ ]:
len(feature_names)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor.fit(X_train_enc, y_train)

In [ ]:
importances = regressor.feature_importances_
std = np.std(
    [
        tree.feature_importances_
        for tree in regressor.estimators_
    ],
    axis=0
)
indices = np.argsort(importances)
# Sort from least to most
indices = list(reversed(indices))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 9))
plt.title("Feature importances")
n = 20
n_indices = indices[:n]
labels = np.array(feature_names)[n_indices]
plt.barh(range(n), importances[n_indices], color="b", yerr=std[n_indices])
plt.yticks(range(n), labels, size=15)
plt.tight_layout(pad=1)
plt.show()

### From scratch stuff